---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

import pandas as pd
import numpy as np

df = pd.read_csv('train.csv', encoding='ISO-8859-1')
df.head()

# Exploring the dataset

## agency_name

Only 5 agencies are seen in training set.

However, they are very unvevenly distributed AND new agencies may appear in the future.

Let us **NOT INCLUDE** this feature in our model.

df['agency_name'].value_counts()

## violator_name

* There are some repeated offenders in the dataset
* MAYBE engineer a repeated_offence feature 
* complication: need to pre-process the TEST set as well
* complication2: vast majority are non-repeat offenders
* some have 2-3 offences, others have hundreds
* possible missplellings split ONE offender into multiple

Let us **NOT INCLUDE** this feature in our model;


**POSSIBLE TO RECONSIDER**

print("""{:,} violators received {:,} tickets.
Average of {:0.0f} tickets per violator""".format(len(df['violator_name'].unique()), 
                                             len(df),
                                             len(df)/len(df['violator_name'].unique())
                                            )
     )

df['violator_name'].value_counts()

# Location:

There may exist unhospitable areas that do not pay fines; Try to explore that angle

* violation_street_number - ** DO NOT INCLUDE **
* violation_street_name  - ** DO NOT INCLUDE **
* violation_zip_code - ** ALL NANs**

detroit_spellings = ['DETROIT', 'DEROIT','DEROT','DERROIT','DERTOIT','DERTROIT','DET','DET48234','DETAROIT','DETEOIT','DETEROIT','DETOIT','DETORIIT','DETORIT','DETR4OIT','DETREOIT','DETRIOIT','DETRIOT','DETRIT','DETRIUT','DETRJOIT','DETROFIT','DETROI','DETROIIT','DETROIOT','DETROIR','DETROIRT','DETROIS','DETROIT','DETROIT1','DETROIT4','DETROITDETROIT','DETROITF','DETROITI','DETROITL','DETROITLI','DETROITM','DETROITMI','DETROITMI48206','DETROITQ','DETROITT','DETROITY','DETROIYT','DETROKT','DETROOIT','DETROPIT','DETRORIT','DETROT','DETROTI','DETROUIT','DETRPOT','DETRROIT','DETRTOI','DETRTOIT','DETTROIT','DETTROITM','DETYROIT']
len(detroit_spellings)

detroit_spellings = ['DETROIT', 'DEROIT','DEROT','DERROIT','DERTOIT','DERTROIT','DET','DET48234','DETAROIT','DETEOIT','DETEROIT','DETOIT','DETORIIT','DETORIT','DETR4OIT','DETREOIT','DETRIOIT','DETRIOT','DETRIT','DETRIUT','DETRJOIT','DETROFIT','DETROI','DETROIIT','DETROIOT','DETROIR','DETROIRT','DETROIS','DETROIT','DETROIT1','DETROIT4','DETROITDETROIT','DETROITF','DETROITI','DETROITL','DETROITLI','DETROITM','DETROITMI','DETROITMI48206','DETROITQ','DETROITT','DETROITY','DETROIYT','DETROKT','DETROOIT','DETROPIT','DETRORIT','DETROT','DETROTI','DETROUIT','DETRPOT','DETRROIT','DETRTOI','DETRTOIT','DETTROIT','DETTROITM','DETYROIT']

def clean_str_row(s,
                   normalize_case=True,
                   remove_spaces=False,
                   remove_punctuation=False,
                   spellings=[]
                  ):
    assert type(s) is str
    assert type(spellings) is list
    
    

    if normalize_case:
        s = s.upper()
    if remove_punctuation:
        exclude = set(string.punctuation)
        s = ''.join(ch for ch in s if ch not in exclude)
    if remove_spaces:
        exclude = set(' ')
        s = ''.join(ch for ch in s if ch not in exclude)

    if s in spellings:
        return 'DETROIT'
    else:
        return s
    
df['city'] = df['city'].apply(clean_str_row)

df_detroit = df[df['city'] == 'DETROIT']
df_detroit

len(df['state'].value_counts())

df['state'].value_counts().loc['MI']/len(df['state'].dropna())

91% of all violations were by firms registered in Michigan;

**MAYBE** include **`state`** as a feature;
Same with **`zip_code`**

df_detroit['zip_code'].isnull().sum()

pd.merge(df_detroit, pd.get_dummies(df_detroit['zip_code'].apply(lambda x: 'zip_'+str(x))), left_index=True, right_index=True)

df_detroit['zip_code'].value_counts()[:10]

## `ticket_issued_date` - Date and time the ticket was issued
## `hearing_date` - Date and time the violator's hearing was scheduled


**Not usable**

## violation_code, violation_description - Type of violation

** preprocess `violation_code` to unite related codes

** INCLUDE `violation_code` feature (as categorical) **

for i, frame in df[['violation_code','violation_description']].drop_duplicates().sort_values(['violation_code']).iterrows():
    print(frame['violation_code'], ' | ', frame['violation_description'][:100])
    print('='*110)

def clean_violation_code(s):
    assert type(s) is str, 'Func only takes STRING'
    
    import re
    return re.split(' |\(|/', s)[0]

for i, frame in df[['violation_code','violation_description']].drop_duplicates().sort_values(['violation_code']).iterrows():
    print(clean_violation_code(frame['violation_code']),
          ' | ',
          frame['violation_description'][:90]
         )
    print('='*110)
    
df['violation_code'] = df['violation_code'].apply(clean_violation_code)

## disposition - Judgment and judgement type

Feature relating to outcome, **DO NOT INCLUDE** due to information leaking

df['disposition'].value_counts()

## fine_amount - Violation fine amount, excluding fees

Continuous value, **INCLUDE FEATURE**


df['fine_amount'].describe()

df['fine_amount'].isnull().sum()

df['admin_fee']

## admin_fee  usd 20 fee assigned to responsible judgments
## state_fee - usd 10 fee assigned to responsible judgments
## late_fee - 10% fee assigned to responsible judgments

Not sure how these features work;

Suspect Data Leakage: ** DO NOT INCLUDE **

** INVESTIGATE FURTHER **

## discount_amount - discount applied, if any

** INCLUDE FEATURE **, continuous value

df['discount_amount'][df['discount_amount'] > 0].describe()

df['discount_amount'].isnull().sum()

## clean_up_cost - DPW clean-up or graffiti removal cost

All is zero --> ** DO NOT INCLUDE FEATURE **

df['clean_up_cost'].sum()

## judgment_amount - Sum of all fines and fees

df['judgment_amount'].iloc[:10]

## grafitti_status - Flag for graffiti violations

Only *1* row not na; `do not include feature`

len(df['grafitti_status'].dropna())

## List all `features`


* ticket_id - unique identifier for tickets

* agency_name - Agency that issued the ticket

* inspector_name - Name of inspector that issued the ticket

* violator_name - Name of the person/organization that the ticket was issued to

* violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred

* mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator

* ticket_issued_date - Date and time the ticket was issued

* hearing_date - Date and time the violator's hearing was scheduled

* violation_code, violation_description - Type of violation

* disposition - Judgment and judgement type

* fine_amount - Violation fine amount, excluding fees

* admin_fee - $\$$20 fee assigned to responsible judgments

* state_fee - $\$$10 fee assigned to responsible judgments

* late_fee - 10$\%$ fee assigned to responsible judgments

* discount_amount - discount applied, if any

* clean_up_cost - DPW clean-up or graffiti removal cost

* judgment_amount - Sum of all fines and fees

* grafitti_status - Flag for graffiti violations

## List of **chosen** features

* zip_code

* violation_code

## List of ignored features

import pandas as pd
import numpy as np

df = pd.read_csv('train.csv', encoding='ISO-8859-1')

detroit_spellings = ['DETROIT', 'DEROIT','DEROT','DERROIT','DERTOIT','DERTROIT','DET','DET48234','DETAROIT','DETEOIT','DETEROIT','DETOIT','DETORIIT','DETORIT','DETR4OIT','DETREOIT','DETRIOIT','DETRIOT','DETRIT','DETRIUT','DETRJOIT','DETROFIT','DETROI','DETROIIT','DETROIOT','DETROIR','DETROIRT','DETROIS','DETROIT','DETROIT1','DETROIT4','DETROITDETROIT','DETROITF','DETROITI','DETROITL','DETROITLI','DETROITM','DETROITMI','DETROITMI48206','DETROITQ','DETROITT','DETROITY','DETROIYT','DETROKT','DETROOIT','DETROPIT','DETRORIT','DETROT','DETROTI','DETROUIT','DETRPOT','DETRROIT','DETRTOI','DETRTOIT','DETTROIT','DETTROITM','DETYROIT']

def clean_str_row(s,
                   normalize_case=True,
                   remove_spaces=False,
                   remove_punctuation=False,
                   spellings=[]
                  ):
    assert type(s) is str
    assert type(spellings) is list
    
    

    if normalize_case:
        s = s.upper()
    if remove_punctuation:
        exclude = set(string.punctuation)
        s = ''.join(ch for ch in s if ch not in exclude)
    if remove_spaces:
        exclude = set(' ')
        s = ''.join(ch for ch in s if ch not in exclude)

    if s in spellings:
        return 'DETROIT'
    else:
        return s
    
df['city'] = df['city'].apply(clean_str_row)

# train only on Detroit
df_detroit = df[df['city'] == 'DETROIT']

# remove tickets that were cancelled
df_detroit = df_detroit[df_detroit['compliance'].notnull()]

def clean_violation_code(s):
    assert type(s) is str, 'Func only takes STRING'
    
    import re
    return re.split(' |\(|/', s)[0]

# convert categorical data in df[colname] to columns
def col_to_dummy(df, colname):
    assert type(df) is pd.DataFrame, 'type error'
    assert type(colname) is str, 'type error'
    
    # add zip_code dummies
    df = pd.merge(df, pd.get_dummies(df[colname].apply(lambda x: colname+'_'+str(x))), left_index=True, right_index=True)
    df = df.drop(colname, axis=1)
    return df

def prep_dataframe_for_model(df):

    feature_names = ['agency_name',
                       #'zip_code',
                       'violation_code',
                       'fine_amount',
                       'admin_fee',
                       'state_fee',
                       'late_fee'
                        ]
    df.index = df['ticket_id']
    df = df[feature_names]
    df['violation_code'] = df['violation_code'].apply(clean_violation_code)
    
    categorical = ['agency_name', 'violation_code'] #'zip_code'

    for col in categorical:
        df = col_to_dummy(df, col)
        
        
    return df
        
X_fines = prep_dataframe_for_model(df_detroit)
y_fines = df_detroit['compliance'].copy()
#y_fines = df_detroit['compliance']
X_fines.head(2)

X_predict = prep_dataframe_for_model(pd.read_csv('test.csv',encoding='ISO-8859-1'))
X_predict.head()

# features that exist BOTH in train and test
common_features = sorted((set(X_predict.columns) & set(X_fines.columns)))
X_fines=X_fines[common_features]
X_predict=X_predict[common_features]

# normalize continuous values

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_fines_scaled = scaler.fit_transform(X_fines)
X_predict_scaled = scaler.transform(X_predict)
X_fines_scaled

from sklearn.model_selection import train_test_split

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_fines_scaled,
                                                                  y_fines,
                                                                  random_state = 0)

%%time

from matplotlib import cm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc


lr = LogisticRegression()

y_score_lr = lr.fit(X_train_scaled, y_train).decision_function(X_test_scaled)
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)

plt.figure()
plt.xlim([-0.01, 1.00])
plt.ylim([-0.01, 1.01])
plt.plot(fpr_lr, tpr_lr, lw=3, label='LogRegr ROC curve (area = {:0.2f})'.format(roc_auc_lr))
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
plt.legend(loc='lower right', fontsize=13)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.axes().set_aspect('equal')
plt.show()


plt.figure()
plt.xlim([-0.01, 1.00])
plt.ylim([-0.01, 1.01])

for c in [0.01, 0.1, 0.20, 1, 5, 10, 100, 1000]:
    lr = LogisticRegression(C=c).fit(X_train_scaled, y_train)
    y_score_lr = lr.decision_function(X_test_scaled)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    accuracy_lr = lr.score(X_test_scaled, y_test)
    print("C = {:.2f}  accuracy = {:.2f}   AUC = {:.2f}".format(c, accuracy_lr, 
                                                                    roc_auc_lr))
    plt.plot(fpr_lr, tpr_lr, lw=3, alpha=0.7, 
             label='LR (gamma = {:0.2f}, area = {:0.2f})'.format(c, roc_auc_lr))

plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate (Recall)', fontsize=16)
plt.plot([0, 1], [0, 1], color='k', lw=0.5, linestyle='--')
plt.legend(loc="lower right", fontsize=11)
plt.title('ROC curve: (fines classifier)', fontsize=16)
plt.axes().set_aspect('equal')

plt.show()

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    """
    For this assignment, create a function that trains a model to predict blight ticket compliance 
    in Detroit using train.csv. Using this model, return a series of length 61001 with the data being 
    the probability that each corresponding ticket from test.csv will be paid, and the index being 
    the ticket_id."""
    # Your code here
    import pandas as pd
    import numpy as np

    df = pd.read_csv('train.csv', encoding='ISO-8859-1')

    detroit_spellings = ['DETROIT', 'DEROIT','DEROT','DERROIT','DERTOIT','DERTROIT','DET','DET48234','DETAROIT','DETEOIT','DETEROIT','DETOIT','DETORIIT','DETORIT','DETR4OIT','DETREOIT','DETRIOIT','DETRIOT','DETRIT','DETRIUT','DETRJOIT','DETROFIT','DETROI','DETROIIT','DETROIOT','DETROIR','DETROIRT','DETROIS','DETROIT','DETROIT1','DETROIT4','DETROITDETROIT','DETROITF','DETROITI','DETROITL','DETROITLI','DETROITM','DETROITMI','DETROITMI48206','DETROITQ','DETROITT','DETROITY','DETROIYT','DETROKT','DETROOIT','DETROPIT','DETRORIT','DETROT','DETROTI','DETROUIT','DETRPOT','DETRROIT','DETRTOI','DETRTOIT','DETTROIT','DETTROITM','DETYROIT']

    def clean_str_row(s,
                       normalize_case=True,
                       remove_spaces=False,
                       remove_punctuation=False,
                       spellings=[]
                      ):
        assert type(s) is str
        assert type(spellings) is list



        if normalize_case:
            s = s.upper()
        if remove_punctuation:
            exclude = set(string.punctuation)
            s = ''.join(ch for ch in s if ch not in exclude)
        if remove_spaces:
            exclude = set(' ')
            s = ''.join(ch for ch in s if ch not in exclude)

        if s in spellings:
            return 'DETROIT'
        else:
            return s

    df['city'] = df['city'].apply(clean_str_row)

    # train only on Detroit
    df_detroit = df[df['city'] == 'DETROIT']

    # remove tickets that were cancelled
    df_detroit = df_detroit[df_detroit['compliance'].notnull()]
    
    X_fines   = prep_dataframe_for_model(df_detroit)
    X_predict = prep_dataframe_for_model(pd.read_csv('test.csv',encoding='ISO-8859-1'))
    
    # features that exist BOTH in train and test
    common_features = sorted((set(X_predict.columns) & set(X_fines.columns)))
    X_fines=X_fines[common_features]
    X_predict=X_predict[common_features]

    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()

    X_fines_scaled = scaler.fit_transform(X_fines)
    X_predict_scaled = scaler.transform(X_predict)
    X_fines_scaled
    
    from sklearn.model_selection import train_test_split

    X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_fines_scaled,
                                                                      y_fines,
                                                                      random_state = 0)
    lr = LogisticRegression(C=10).fit(X_train_scaled, y_train)
    return pd.Series(lr.predict_proba(X_predict_scaled)[:, 1], # only return column corresponding to prob Y=1
                     index=X_predict.index) 

blight_model()

/home/dobrinskiy/miniconda3/envs/tf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2808: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


NameError: name 'prep_dataframe_for_model' is not defined